In [1]:
import pandas as pd
import os
import numpy as np
import cv2
from utils import load_detections, track_iou, create_similarity_matrix, associate_detections_to_tracks, update_tracks, draw_tracking_result, save_tracking_results

###  Loading det object instances

In [2]:
det_file_path = '../data/det/det.txt'
detections_df = load_detections(det_file_path)
detections_df.head()

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.567,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,29.439,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,19.601,-1,-1,-1
3,1,-1,31,525,113.370,257.270,17.013,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,11.949,-1,-1,-1


In [3]:
tracks = []
frame_number = 1
sigma_iou = 0.5

#### IoU for tracking

In [4]:
# Example with frame 1 and 2
# Create previous tracks
previous_tracks = []
for i, row in detections_df[detections_df['frame'] == 1].iterrows():
    center = (int(row['bb_left'] + row['bb_width'] / 2), int(row['bb_top'] + row['bb_height'] / 2))
    previous_tracks.append({
        'id': row['id'],
        'frames': [row['frame']],
        'box': [row['bb_left'], row['bb_top'], row['bb_width'], row['bb_height']],
        'conf': [row['conf']],
        'positions': [center]
    })

# Create current detections
current_detections = []
for i, row in detections_df[detections_df['frame'] == 2].iterrows():
    current_detections.append([row['bb_left'], row['bb_top'], row['bb_width'], row['bb_height']])

# Create similarity matrix
similarity_matrix = create_similarity_matrix(current_detections, previous_tracks)
similarity_matrix

array([[1.        , 0.        , 0.        , 0.        , 0.12448738],
       [0.        , 0.74473154, 0.        , 0.        , 0.        ],
       [0.36184537, 0.        , 0.        , 0.        , 0.35833176],
       [0.        , 0.0219888 , 0.54437704, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.6335195 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

#### Associate detections to tracks

In [5]:
# Example with frame 1 and 2
matches, unmatched_detections, unmatched_tracks = associate_detections_to_tracks(similarity_matrix, sigma_iou)
matches, unmatched_detections, unmatched_tracks

([(0, 0), (1, 1), (3, 2), (4, 3)], [2, 5], [4])

#### Track management

In [6]:
# Example with frame 1 and 2
# Update tracks
current_boxes = []
for i, row in detections_df[detections_df['frame'] == 2].iterrows():
    current_boxes.append([row['bb_left'], row['bb_top'], row['bb_width'], row['bb_height']])
current_conf = detections_df[detections_df['frame'] == 2]['conf'].tolist()

tracks = update_tracks(matches, unmatched_tracks, unmatched_detections, current_boxes, current_conf, previous_tracks, 2)
tracks

[{'id': -1.0,
  'frames': [1.0, 2],
  'box': [1689.0, 385.0, 146.62, 332.71],
  'conf': 66.725,
  'positions': [(1762, 551), (1762, 551)]},
 {'id': -1.0,
  'frames': [1.0, 2],
  'box': [1312.0, 503.0, 61.514, 139.59],
  'conf': 36.614,
  'positions': [(1333, 572), (1342, 572)]},
 {'id': -1.0,
  'frames': [1.0, 2],
  'box': [1254.0, 537.0, 52.0, 118.0],
  'conf': 15.979,
  'positions': [(1278, 614), (1280, 596)]},
 {'id': -1.0,
  'frames': [1.0, 2],
  'box': [55.0, 542.0, 94.66, 214.81],
  'conf': 9.3326,
  'positions': [(87, 653), (102, 649)]},
 {'id': 1,
  'box': [1744.0, 476.0, 123.42, 280.06],
  'conf': 16.976,
  'frames': [2],
  'positions': [(1805, 616)]},
 {'id': 2,
  'box': [1641.0, 494.0, 40.123, 91.049],
  'conf': 6.0638,
  'frames': [2],
  'positions': [(1661, 539)]}]

#### Interface (main loop)

In [7]:
# Main loop
image_frames_path = '../data/img1/' 

# Define properties for the output video
fps = 30
frame_size = (1920, 1080)  # Adjust this to match the size of your frames
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
out = cv2.VideoWriter('output_with_tracking.mp4', fourcc, fps, frame_size)
delay = int(1000/fps)

# Initialize tracking variables
tracks = []
frame_number = 1
sigma_iou = 0.2

tracking_file = 'track.txt'
if os.path.isfile(tracking_file):
    open(tracking_file, 'w').close()
else:
    open(tracking_file, 'x')

# Process each image frame
for filename in sorted(os.listdir(image_frames_path)):
    if filename.endswith(".jpg"):
        frame_path = os.path.join(image_frames_path, filename)
        frame = cv2.imread(frame_path)

        # Get current detections for this frame
        current_detections_df = detections_df[detections_df['frame'] == frame_number]
        current_boxes = current_detections_df[['bb_left', 'bb_top', 'bb_width', 'bb_height']].values
        current_confidences = current_detections_df['conf'].values

        # Compute similarity matrix and associate detections to tracks
        if frame_number > 1:
            similarity_matrix = create_similarity_matrix(current_boxes, tracks)
            matches, unmatched_detections, unmatched_tracks = associate_detections_to_tracks(similarity_matrix, sigma_iou=0.5)
            tracks = update_tracks(matches, unmatched_tracks, unmatched_detections, current_boxes, current_confidences, tracks, frame_number)
        else:
            for i, row in current_detections_df.iterrows():
                initial_center = (int(row['bb_left'] + row['bb_width'] / 2), int(row['bb_top'] + row['bb_height'] / 2))
                tracks.append({
                    'id': row['id'],
                    'box': [row['bb_left'], row['bb_top'], row['bb_width'], row['bb_height']],
                    'conf': row['conf'],
                    'frames': [frame_number],
                    'positions': [initial_center]
                })

        # print(f'Frame {frame_number}: {len(tracks)} tracks, {len(unmatched_detections)} unmatched detections, {len(unmatched_tracks)} unmatched tracks, {len(matches)} matches')
        
        # Draw tracking
        frame_with_tracking = draw_tracking_result(frame, tracks)
        # Save the tracking results
        save_tracking_results(tracks, tracking_file, frame_number)

        # Show the frame
        cv2.imshow('Tracking', frame_with_tracking)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
        out.write(frame_with_tracking)

        frame_number += 1

# Release everything when done
out.release()
cv2.destroyAllWindows()